In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook, tnrange
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
PB = ProgressBar()
PB.register()

In [3]:
leave = dd.read_csv("data/rt_leavetimes_DB_2018.txt", sep=";")
trips = pd.read_csv("data/rt_trips_DB_2018.txt", sep=";")

In [ ]:
leave = leave.drop(['DATASOURCE','VEHICLEID','PASSENGERS','PASSENGERSIN','PASSENGERSOUT','DISTANCE','SUPPRESSED', 'JUSTIFICATIONID','NOTE','LASTUPDATE'], axis=1)
leave['DAYOFSERVICE'] = dd.to_datetime(leave.DAYOFSERVICE, format="%d-%b-%y %H:%M:%S")

In [ ]:
trips = trips.drop(['DATASOURCE','BASIN','TENDERLOT','SUPPRESSED','JUSTIFICATIONID','NOTE','LASTUPDATE','PLANNEDTIME_ARR','PLANNEDTIME_DEP','ACTUALTIME_ARR','ACTUALTIME_DEP'], axis=1)
trips['DAYOFSERVICE'] = pd.to_datetime(trips.DAYOFSERVICE, format="%d-%b-%y %H:%M:%S")

In [ ]:
combine = leave.merge(trips, how='left', on=['DAYOFSERVICE','TRIPID'])

In [ ]:
combine = combine[['DAYOFSERVICE','TRIPID','LINEID','DIRECTION','PROGRNUMBER','STOPPOINTID','PLANNEDTIME_DEP','PLANNEDTIME_ARR','ACTUALTIME_DEP','ACTUALTIME_ARR','ROUTEID']]

In [ ]:
combinedf = combine.compute()

[#####                                   ] | 12% Completed |  1min 21.9s

In [ ]:
combinedf.head()

In [ ]:
combinedf.dtypes

In [4]:
combinedf.count()

In [17]:
combinedf.to_csv('combined.csv', index=False, header=None, chunksize=500000)